In [4]:
from mongodb.utils import mongodb_funciones
from google_sheet.utils import *
from utils.utils import *
import pandas as pd

In [5]:
# Crea una instancia de la clase mongodb_funciones
mongo_func = mongodb_funciones()

MongoClient(host=['shared-shard-00-00.eh6ne.mongodb.net:27017', 'shared-shard-00-01.eh6ne.mongodb.net:27017', 'shared-shard-00-02.eh6ne.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, replicaset='atlas-48izhi-shard-0', authsource='admin', tls=True)


In [3]:
# Crea una instancia de la clase mongodb_funciones
google_sheet_func = google_sheet_funciones()

In [4]:
datos_bd = mongo_func.datos_con_aggregation_personalizado("publications")

In [5]:
#google_sheet_func.mostrar_hojas()

In [6]:
hoja = google_sheet_func.seleccionar_hoja('Actualizaciones MKP Regional version 2')

In [7]:
df_marketplace = pd.DataFrame(datos_bd)

In [8]:
df_marketplace_copia = df_marketplace.copy()

Verificar stock

In [9]:
df_marketplace_copia['stock_final'] = df_marketplace_copia['stock'].apply(verificar_stock)

Reordenar columnas

In [10]:
df_marketplace_copia2 = df_marketplace_copia[['country', 'brand', 'categoryCode', 'model', 'year', 'price_base', 'price_net', 'published', 'colors', 'stock', 'stock_final']]

In [11]:
df_marketplace_copia2 = traducir_nombres_columnas(df_marketplace_copia2)

In [12]:
df_marketplace_copia2[df_marketplace_copia2['Precio neto'].isna()]
df_marketplace_copia2['Precio neto'] = df_marketplace_copia2['Precio neto'].fillna(df_marketplace_copia2['Precio Base'])
df_marketplace_copia2['Precio neto'] = df_marketplace_copia2['Precio neto'].astype(int)
# Calculate the difference and create a new column 'Descuento'
df_marketplace_copia2['Descuento'] = df_marketplace_copia2['Precio Base'] - df_marketplace_copia2['Precio neto']

In [13]:
df_marketplace_copia2 = df_marketplace_copia2.drop(columns=["colors", "stock"])

In [14]:
df_marketplace_copia2 = df_marketplace_copia2.rename(columns={"stock_final": "Stock"})

In [15]:
df_marketplace_copia2

,country,Marca,categoryCode,Modelo,Año,Precio Base,Precio neto,Publicado,Stock,Descuento
0,CL,Eve Motors,CL-MOTORCYCLES,JGW,2023,1100000,1100000,True,True,0
1,PE,Semza,PE-MOTORCYCLES,Spark 200,2023,7700,7700,True,False,0
2,MX,Yamaha,MX-MOTORCYCLES,MT 09 2024,2024,299999,299999,True,True,0
3,CL,Loncin,CL-MOTORCYCLES,GA300 PRO,2024,4150000,3990000,True,True,160000
4,CL,Kymco,CL-MOTORCYCLES,Agility RS 125,2024,1599900,1589900,True,False,10000
...,...,...,...,...,...,...,...,...,...,...
2481,MX,Vento,MX-MOTORCYCLES,Xplor 190 2025,2025,26999,24999,True,True,2000
2482,PE,Zongshen,PE-MOTORCYCLES,Evo 125,2023,5107,4864,True,False,243
2483,MX,Bajaj,MX-MOTORCYCLES,Pulsar N 250 2023,2023,59999,59999,True,False,0
2484,CO,Super Soco,CO-MOTORCYCLES,Cux 2022,2022,8999000,8999000,True,True,0


In [16]:
# Definir el nuevo orden de columnas
nuevo_orden = ["country", 'categoryCode', 'Marca', 'Modelo', 'Año', 'Precio Base', 'Precio neto', 'Descuento', 'Publicado', 'Stock']

# Reordenar el DataFrame
df_marketplace_copia2 = df_marketplace_copia2[nuevo_orden]

### Base completa

In [17]:
df_marketplace["categoryCode"].unique()

array(['CL-MOTORCYCLES', 'PE-MOTORCYCLES', 'MX-MOTORCYCLES',
       'CO-MOTORCYCLES', 'CL-CARS', 'CO-CARS'], dtype=object)

### Separar por paises

México

In [18]:
df_marketplace_copia_mx_motos = df_marketplace_copia2[(df_marketplace_copia2['country'] == "MX") & (df_marketplace_copia2['categoryCode'] == "MX-MOTORCYCLES")]

Chile

In [19]:
df_marketplace_copia_cl_motos = df_marketplace_copia2[(df_marketplace_copia2['country'] == "CL") & (df_marketplace_copia2['categoryCode'] == "CL-MOTORCYCLES")]

In [20]:
df_marketplace_copia_cl_autos = df_marketplace_copia2[(df_marketplace_copia2['country'] == "CL") & (df_marketplace_copia2['categoryCode'] == "CL-CARS")]

Colombia

In [21]:
df_marketplace_copia_co_motos = df_marketplace_copia2[(df_marketplace_copia2['country'] == "CO") & (df_marketplace_copia2['categoryCode'] == "CO-MOTORCYCLES")]

Perú

In [22]:
df_marketplace_copia_pe_motos = df_marketplace_copia2[(df_marketplace_copia2['country'] == "PE") & (df_marketplace_copia2['categoryCode'] == "PE-MOTORCYCLES")]

In [23]:
df_marketplace_copia_pe_motos

,country,categoryCode,Marca,Modelo,Año,Precio Base,Precio neto,Descuento,Publicado,Stock
1,PE,PE-MOTORCYCLES,Semza,Spark 200,2023,7700,7700,0,True,False
5,PE,PE-MOTORCYCLES,Ssenda,Patagonia 250 V8,2023,13990,13990,0,True,False
7,PE,PE-MOTORCYCLES,Semza,Tank,2023,5600,5600,0,True,False
16,PE,PE-MOTORCYCLES,Bruno,250 RR,2023,8700,8700,0,True,False
20,PE,PE-MOTORCYCLES,Suzuki,GSX 250 FI,2023,20136,20136,0,True,False
...,...,...,...,...,...,...,...,...,...,...
2460,PE,PE-MOTORCYCLES,Kawasaki,Z900,2023,70000,70000,0,True,False
2463,PE,PE-MOTORCYCLES,Bruno,125 CG,2023,3600,3600,0,True,False
2473,PE,PE-MOTORCYCLES,TVS,Stryker,2022,5499,5499,0,True,False
2475,PE,PE-MOTORCYCLES,Bruno,125 T1,2023,4600,4600,0,True,False


In [24]:
df_marketplace_copia_cl_motos

,country,categoryCode,Marca,Modelo,Año,Precio Base,Precio neto,Descuento,Publicado,Stock
0,CL,CL-MOTORCYCLES,Eve Motors,JGW,2023,1100000,1100000,0,True,True
3,CL,CL-MOTORCYCLES,Loncin,GA300 PRO,2024,4150000,3990000,160000,True,True
4,CL,CL-MOTORCYCLES,Kymco,Agility RS 125,2024,1599900,1589900,10000,True,False
6,CL,CL-MOTORCYCLES,KTM,RC 200,2023,4150000,3390000,760000,True,True
18,CL,CL-MOTORCYCLES,Yamaha,FZ X,2023,2699000,2290000,409000,True,True
...,...,...,...,...,...,...,...,...,...,...
2439,CL,CL-MOTORCYCLES,Kawasaki,Z900 ABS,2023,12590000,12590000,0,True,False
2445,CL,CL-MOTORCYCLES,UM,Renegade Freedom 300,2023,4290000,4290000,0,True,True
2464,CL,CL-MOTORCYCLES,Daas Motors,LX 04,2023,1599000,1589000,10000,True,True
2474,CL,CL-MOTORCYCLES,Zongshen,Cyclone RA2,2024,2899900,2599900,300000,True,True


Renombre de columna "country"

In [25]:
# Diccionario de mapeo: {'nombre_actual': 'nuevo_nombre'}
rename_mapping = {"country": "País"}

# Renombrar columnas y eliminar 'categoryCode' en cada DataFrame original
for df in [
    df_marketplace_copia_mx_motos,
    df_marketplace_copia_cl_motos,
    df_marketplace_copia_cl_autos,
    df_marketplace_copia_co_motos,
    df_marketplace_copia_pe_motos
]:
    df.rename(columns=rename_mapping, inplace=True)
    df.drop(columns=["categoryCode"], inplace=True)

C:\Users\Bradl\AppData\Local\Temp\ipykernel_14112\693618495.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns=rename_mapping, inplace=True)
C:\Users\Bradl\AppData\Local\Temp\ipykernel_14112\693618495.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=["categoryCode"], inplace=True)
C:\Users\Bradl\AppData\Local\Temp\ipykernel_14112\693618495.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.r

### Actualización de hojas

In [26]:
# Lista de DataFrames para actualizar múltiples hojas
dataframes_to_update = [
    ("MX", df_marketplace_copia_mx_motos, "Moto"),
    ("CL", df_marketplace_copia_cl_motos, "Moto"),
    ("CL", df_marketplace_copia_cl_autos, "Auto"),
    ("CO", df_marketplace_copia_co_motos, "Moto"),
    ("PE", df_marketplace_copia_pe_motos, "Moto")
]


In [27]:
google_sheet_func.actualizar_todas_las_hojas(hoja, dataframes_to_update)

Updated sheet: Base MX 03/11 Moto
Updated sheet: Base CL 03/11 Moto
Updated sheet: Base CL 03/11 Auto
Updated sheet: Base CO 03/11 Moto
Updated sheet: Base PE 03/11 Moto


In [28]:
# Actualizar la hoja específica
google_sheet_func.actualizar_hoja_base(hoja, df_marketplace)

Updated sheet: Base total 03/11
